# Summary

In [1]:
from abstractor.train import get_training_batch as get_abstractor_training_batch
from abstractor.utils import AbstractorModel, AbstractorModelRNN
from abstractor.utils import obtain_initial_hidden_states
from bert.utils import obtain_sentence_embeddings
from bert.utils import obtain_word_embeddings
from data.utils import load_training_dictionaries
from extractor.train import get_training_batch as get_extractor_training_batch
from extractor.utils import ExtractorModel
from pytorch_transformers import BertModel
from pytorch_transformers import BertTokenizer
from rl_connection.utils import RLModel
from rl_connection.train import get_training_batch as get_rl_training_batch
from rouge import Rouge

import numpy as np
import torch

## Load data

In [2]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [3]:
data = load_training_dictionaries()

## Extractor

In [4]:
# Load extractor model:
extractor_model = ExtractorModel(bert_tokenizer, bert_model)
extractor_model_path = "results/models/extractor.pt"
extractor_model.load_state_dict(torch.load(extractor_model_path))

<All keys matched successfully>

In [5]:
documents, extraction_labels = get_extractor_training_batch(data, batch_size=2)

sentence_embeddings, mask = obtain_sentence_embeddings(
    extractor_model.bert_model, 
    extractor_model.bert_tokenizer, 
    documents
)

# Predict probability of extraction per sentence
extraction_probabilities = extractor_model(sentence_embeddings)

In [6]:
n_samples = len(documents)

for sample_idx in range(n_samples):
    n_to_extract = extraction_labels.sum(dim=1)[sample_idx].int() 
    ext_prob = extraction_probabilities[sample_idx] * mask[sample_idx]
    ext_sent_indicies = torch.topk(ext_prob, k=n_to_extract)[1]
    
    targets = np.array(documents[sample_idx])[extraction_labels[sample_idx][:len(documents[sample_idx])].numpy().astype(bool)]
    print("----> TARGET <----")
    for target in targets:
        print(f"{target}\n")
    print()
          
    print("----> PREDICTION <----")
    for x in np.array(documents[sample_idx])[ext_sent_indicies]:
        print(f"{x} \n")
    print("\n\n-------\n\n")

----> TARGET <----
" weasels will go for anything that looks like food -- they 've got a high metabolism and they 've got to eat a lot , " she said . " it does n't surprise me that a weasel took a punt -- i 've seen a photo of a weasel charging a group of sparrows , they 're very hungry animals . "

weasels would not normally target green woodpeckers , pacheco said -- their predators are normally the size of a stoat or larger . but the birds are known to spend a fair amount of time on the ground pulling up worms and hunting insects .

the pluckiness of the weasel spawned a number of parodies on twitter , with manipulated images showing the creature in turn being ridden by russian president vladimir putin , popstar miley cyrus , football star john terry -- and even what appears to be a dog red panda dressed in a darth vader costume . ( update : twitter has now educated us on the difference between a dog and a red panda . sorry , darth ! )


----> PREDICTION <----
the pluckiness of the w

## Abstractor

In [7]:
# Load data:
abstractor_model = AbstractorModelRNN(bert_tokenizer, bert_model)
abstractor_model_path = "results/models/abstractor.pt"
abstractor_model.load_state_dict(torch.load(abstractor_model_path))

<All keys matched successfully>

In [8]:
source_documents, target_summaries = get_abstractor_training_batch(data, 1)

# Obtain embeddings
source_document_embeddings, source_mask, source_tokens = obtain_word_embeddings(
    abstractor_model.bert_model, abstractor_model.bert_tokenizer, source_documents, static_embeddings=False
)
target_summary_embeddings, target_mask, target_tokens = obtain_word_embeddings(
    abstractor_model.bert_model, abstractor_model.bert_tokenizer, target_summaries, static_embeddings=True
)

print(source_documents)
print()
print(target_summaries)

[['one french citizen , one belgian and three malians were killed in the attack in the capital of bamako , said gabriel toure , director of a local hospital .', 'a north african jihadist group , al - murabitun , claimed responsibility for the attack , according to mauritanian news agency al akhbar . the purported claim came in an audio message in which the group said it carried out the attack in retaliation for the killing of one of its leaders , al akhbar said .', '" al - murabitun may be considered a regional competitor to al - qaeda in the islamic maghreb ( aqim ) , " according to the jamestown foundation , a washington - based research and analysis firm . the u.s. state department said in january that al - murabitun is a " newly - formed " militant group that has presence in northern mali .']]

[['a jihadist group claims responsibility in an audio recording , news agency reports .', 'the malian government calls the shooting a " terrorist act "', 'one french citizen , one belgian an

In [9]:
# Obtain extraction probability for each word in vocabulary
extraction_probabilities, teacher_forcing = abstractor_model(
    source_document_embeddings,
    target_summary_embeddings,
    teacher_forcing_pct=0
)  # (batch_size, n_target_words, vocab_size)

vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

for x in [abstractor_model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
    print(f"{x}")
    print("")
    
print("------------")


# Obtain extraction probability for each word in vocabulary
extraction_probabilities, teacher_forcing = abstractor_model(
    source_document_embeddings,
    target_summary_embeddings,
    teacher_forcing_pct=1
)  # (batch_size, n_target_words, vocab_size)

vals, predicted_idx = torch.topk((extraction_probabilities), k=1, dim=2)

for x in [abstractor_model.bert_tokenizer.convert_ids_to_tokens(p) for p in predicted_idx.squeeze().tolist()]:
    print(f"{x}")
    print("")

##ist

group

claims

responsibility

in

an

audio

recording

,

the

mali

##an

government

calls

the

shooting

a

"

terrorist

one

french

citizen

,

one

belgian

and

three

mali

##ans

are

killed

.

[SEP]

three

killed

.

[SEP]

three

killed

.

[SEP]

three

------------
##ist

jihad

##ist

group

claims

responsibility

in

an

audio

recording

,

the

agency

reports

.

the

mali

##an

government

calls

the

shooting

a

"

terrorist

one

"

one

french

citizen

,

one

belgian

and

three

mali

##ans

are

killed

.

[SEP]

three



## Reinforcement Learning

In [10]:
rl_model = RLModel(extractor_model, abstractor_model)
rl_model.load_state_dict(torch.load("results/models/rl.pt"))

<All keys matched successfully>

In [21]:
source_documents, target_summaries = get_rl_training_batch(data, batch_size=1)

# Obtain embeddings
source_sentence_embeddings, source_mask = obtain_sentence_embeddings(
    rl_model.extractor_model.bert_model,
    rl_model.extractor_model.bert_tokenizer,
    source_documents
)
stop_action_index = source_sentence_embeddings.shape[1]
target_summary_embeddings, target_mask, target_tokens = obtain_word_embeddings(
    rl_model.abstractor_model.bert_model,
    rl_model.abstractor_model.bert_tokenizer,
    target_summaries,
    static_embeddings=True
)

In [22]:
# Run trajectory
actions, log_probs, entropys, values, n_ext_sents = rl_model.sample_actions(
    source_sentence_embeddings,
    source_mask
)

# Obtain abstracted sentence from abstractor
predicted_tokens, word_probabilities = rl_model.create_abstracted_sentences(
    actions,
    source_documents,
    n_ext_sents=n_ext_sents,
    teacher_forcing_pct=0,
    target_summary_embeddings=target_summary_embeddings
)
        

In [23]:
# Look at extractions
for art_idx, doc_sentences in enumerate(actions):
    for sent_idx in doc_sentences[:-1]:
        print(source_documents[art_idx][sent_idx])
        print()
    print("\n\n-------\n\n")

this includes promising generation adidas players who enter the mls through the draft systems before completing their college education . homegrown players from club 's development academies are also exempt as are a maximum of three designated players ( dps ) , usually stellar international names whose wages and transfer fees will be covered by club owners or sponsors .

according to phil rawlins , co-primary owner and president of the new mls franchise , orlando city soccer club , " the industry and the game itself has moved on dramatically " in the u.s. . he believes what would equal 50 years growth in most other industries has been experienced in the first two decades of the mls .

this weekend 60,000 fans are expected to witness orlando city 's opening weekend fixture against new york city , another new club making their mls bow . world cup winners kaka and david villa will turn out for orlando and new york city respectively .



-------




In [24]:
# Look at abstractions
for predicted_abstraction in predicted_tokens:
    solution = list()
    for token in predicted_abstraction:
        solution.append(rl_model.abstractor_model.bert_tokenizer.ids_to_tokens[int(token)])
    print(" ".join(solution))
    print("\n\n")

but ##dale ##dale ##dale ##dale ##dale ##dale ##dale ##dale ##dale ##dale ##dale ##dale released released from but was the was the minutes but ##mis later but ##mis later but ##mis later but ##mis





In [25]:
target_summaries

[['the 20th mls season begins this weekend .',
  'league has changed dramatically since its inception in 1996 .',
  'some question whether rules regarding salary caps and transfers need to change .']]